In [1]:
import json 
from openai import OpenAI
import os 

In [2]:
os.environ['OPENAI_API_KEY'] = 'sk-proj-akdeZO2k0thnW5qqSQPiT3BlbkFJFVW45H5G0FVUuzST2rLo'

In [3]:
data = json.load(open('final_result.json'))

In [4]:
data[list(data.keys())[0]]

'Based on the document\'s information, "Item 1. Business" of the Form 10-K for Apple Inc. provides details about the Company\'s business operations, financial condition, and operating results. It discusses the management of the business based on geographic segments such as Americas, Europe, Greater China, Japan, and Rest of Asia Pacific. It also delves into the external factors that can impact Apple Inc.\'s business, including global economic conditions, suppliers, logistics providers, and competitive landscape. For a more in-depth analysis, you can refer to Part II, Item 7, "Management’s Discussion and Analysis of Financial Condition and Results of Operations" of the Form 10-K document. (Page 1-2, Apple Inc. Form 10-K)'

In [5]:
typer = 'Meddic'

In [6]:
prompt = '''
You are a Discovery Question Generator 

- You will be provided with a topic 
- You will be provided with information of that topic 
- You will be provided with a type 

- Your task is to generate a list of discovery questions based on the topic , information and discovery question type seperated by '\n'

Topic : {}

Information : {}

Type : {}
'''

In [7]:
def run_llm(prompt) :

    client = OpenAI()

    stream = client.chat.completions.create(
        model = 'gpt-3.5-turbo' , 
        messages = [
            {
                'role' : 'user' , 
                'content' : prompt}] , 
        stream = True)
    response = ''

    for chunk in stream : 

        if chunk.choices[0].delta.content : response += chunk.choices[0].delta.content

    return response

In [8]:
sample_prompt = prompt.format(
    list(data.keys())[0] , 
    data[list(data.keys())[0]] , 
    'Meddic'
)

In [9]:
response = run_llm(sample_prompt)

In [10]:
response.split('\n')

["What is Apple Inc.'s market segment breakdown by geographic regions?",
 "How does global economic conditions impact Apple Inc.'s business operations?",
 "What are the key external factors that can affect Apple Inc.'s financial condition?",
 'How does Apple Inc. manage its suppliers and logistics providers?',
 "What is Apple Inc.'s competitive landscape like and how does it impact their business strategy?"]

In [11]:
prompt = '''
You are a Question Reframing expert 

- You will be provided with some context 
- You will be provided with a question 
- You will be provided with a Discovery Question Format

- Your task is to reframe the question incorporating context and styling with Discovery Question Format 

Context : {}

Question : {}

Discovery Question Format : Meddic
'''

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [18]:
vc = FAISS.load_local(
    'main_vector_db' , 
    embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large') , 
    allow_dangerous_deserialization = True 
)

In [20]:
similar_docs = vc.similarity_search(response.split('\n')[0])

In [21]:
context = '\n'.join([
    doc.page_content 
    for doc 
    in similar_docs
])

In [22]:
prompt = '''
You are a question framing specialist 

- You will be provided with some context 
- You will be provided with a query 

- Your task is to reframe the query embedding context into the new reframed query 

- Only return the single query and nothing else

Context : {}

Query : {}
'''

In [23]:
prompt = prompt.format(context , response.split('\n')[0])

In [24]:
response = run_llm(prompt)

In [25]:
response

'How does Apple Inc. breakdown its market segments by geographic regions?'